## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tasiilaq,GL,65.6145,-37.6368,24.80,58,75,3.44,broken clouds
1,1,Butaritari,KI,3.0707,172.7902,81.21,82,83,16.71,light rain
2,2,Severo-Kurilsk,RU,50.6789,156.1250,11.59,82,42,5.97,scattered clouds
3,3,Tuktoyaktuk,CA,69.4541,-133.0374,-25.60,75,75,14.97,snow
4,4,Port Alfred,ZA,-33.5906,26.8910,70.00,94,94,16.22,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Butaritari,KI,3.0707,172.7902,81.21,82,83,16.71,light rain
4,4,Port Alfred,ZA,-33.5906,26.8910,70.00,94,94,16.22,overcast clouds
12,12,Mayumba,GA,-3.4320,10.6554,78.98,82,56,3.24,broken clouds
15,15,Namatanai,PG,-3.6667,152.4333,76.98,91,100,13.18,moderate rain
19,19,Carupano,VE,10.6678,-63.2585,79.92,70,64,14.41,broken clouds
22,22,Ulladulla,AU,-35.3500,150.4667,71.01,100,75,11.86,broken clouds
29,29,Atuona,PF,-9.8000,-139.0333,79.50,79,6,18.14,clear sky
30,30,Cururupu,BR,-1.8283,-44.8683,80.02,72,33,5.19,scattered clouds
38,38,Mahebourg,MU,-20.4081,57.7000,78.80,78,40,6.91,light rain
39,39,Bambous Virieux,MU,-20.3428,57.7575,78.80,78,40,6.91,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                149
City                   149
Country                149
Lat                    149
Lng                    149
Max Temp               149
Humidity               149
Cloudiness             149
Wind Speed             149
Current Description    149
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Butaritari,KI,81.21,light rain,3.0707,172.7902,
4,Port Alfred,ZA,70.00,overcast clouds,-33.5906,26.8910,
12,Mayumba,GA,78.98,broken clouds,-3.4320,10.6554,
15,Namatanai,PG,76.98,moderate rain,-3.6667,152.4333,
19,Carupano,VE,79.92,broken clouds,10.6678,-63.2585,
22,Ulladulla,AU,71.01,broken clouds,-35.3500,150.4667,
29,Atuona,PF,79.50,clear sky,-9.8000,-139.0333,
30,Cururupu,BR,80.02,scattered clouds,-1.8283,-44.8683,
38,Mahebourg,MU,78.80,light rain,-20.4081,57.7000,
39,Bambous Virieux,MU,78.80,light rain,-20.3428,57.7575,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame

    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!=""]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Butaritari,KI,81.21,light rain,3.0707,172.7902,Isles Sunset Lodge
4,Port Alfred,ZA,70.00,overcast clouds,-33.5906,26.8910,The Halyards Hotel
12,Mayumba,GA,78.98,broken clouds,-3.4320,10.6554,LIKWALE LODGE
15,Namatanai,PG,76.98,moderate rain,-3.6667,152.4333,Namatanai Hotel
19,Carupano,VE,79.92,broken clouds,10.6678,-63.2585,Hotel La dorada
...,...,...,...,...,...,...,...
400,Port-Gentil,GA,80.60,moderate rain,-0.7193,8.7815,Fengshui Residence
402,Gizo,SB,83.32,light rain,-8.1030,156.8419,Imagination Island
406,Tibati,CM,73.58,scattered clouds,6.4667,12.6333,Tibati Hotel
409,Suleja,NG,86.00,clear sky,9.1758,7.1808,Capy Motel


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [16]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))